In [39]:
import pandas as pd
import datetime
from alice_blue import *

In [40]:
#Get an access token
access_token = AliceBlue.login_and_get_access_token(username='AB063479', password='Ajay143', 
                                                    twoFA='act',
                                                    api_secret=
                                                    'V9TWAT51FBBN5MKTFJKQZ0CTXZTNWK2L9HTFERE8SJ7NU3YLAAJI46W5798L9SE4')
#Create AliceBlue Object, once you have your access_token
#Get master contracts
alice = AliceBlue(username='AB063479', password='Ajay143', access_token=access_token
                  ,master_contracts_to_download=['NSE','NFO','BSE'])

In [42]:
ltp = 0
socket_opened = False

def event_handler_quote_update(message):
    global ltp
    ltp = message['ltp']

def open_callback():
    global socket_opened
    socket_opened = True

In [43]:
socket_opened = False
alice.start_websocket(subscribe_callback=event_handler_quote_update,
                      socket_open_callback=open_callback,
                      run_in_background=True)
while(socket_opened==False):    # wait till socket open & then subscribe
    pass

In [2]:
distance_from_strike = int(input("Please enter the distance value from strike"))
underlying_instrument = str(input("Please enter instrument symbol or token number of instrument"))

Please enter the distance value from strike300
Please enter instrument symbol or token number of instrumentNIFTY


In [54]:
ins_script = alice.get_instrument_by_token('NSE',26000)
alice.subscribe(ins_script, LiveFeedType.COMPACT)

In [74]:
one_minute_ltp = []
while(True):
    if datetime.datetime.today().second % 5 == 0:
        if len(one_minute_ltp) <= 10:
            one_minute_ltp.append(ltp)
        else:
            one_minute_ltp.pop()
    elif len(one_minute_ltp) == 10:
        break

In [83]:
underlying_instrument_ltp = (one_minute_ltp[0] + one_minute_ltp[1] + one_minute_ltp[2]) / 3

In [85]:
rem = underlying_instrument_ltp % 50
if rem % 50 <= 25:
    underlying_instrument_value = underlying_instrument_ltp - rem
else:
    underlying_instrument_value = underlying_instrument_ltp - rem + 50

In [86]:
OTMCall = underlying_instrument_value + 300
OTMPut = underlying_instrument_value - 300

In [81]:
previous_OTMs = (OTMCall,OTMPut)

In [87]:
previous_OTMs

(11500.0, 10900.0)

In [82]:
if curr_OTMS != previous_OTMs:
    buy_signal on prevoius OTMS
    sell signal on current OTMS

(11500.0, 10900.0)